# Jupyter Playbooks for Suricata

* Markus Kont
* Stamus Networks
* github.com/markuskont
* https://twitter.com/markuskont

## Introduction

* Introduce a tool
  * not for experienced data scientists
  * spark some ideas
* Focus on use-cases around Suricata
  * no iris dataset
* Might not have time to cover everything
  * presentation is meant to be a resource

### Fmt.presentation()

 * Presentation **IS** a notebook
 * it is public
 * code examples are live
 * all data is generated by the notebook
   * (from `malware-traffic-analysis.net`)

### whoami

* 2011: Server Administrator
  * Now fully recovered
* 2014: Cyber Security MSc, TalTech
  * 2015: PhD candidate
  * academia was not for me
* 2015: Technology Branch Researcher, NATO CCDCOE
  * trainings, exercises, research
  * met Eric Leblond in 2016
  * later met Peter Manev during Suricon
* 2020: Developer & Threat Researcher, Stamus Networks
  * analytics, suricata rules, detection methods, devops, backend dev, ...
  * ...insert random fancy title here...
  * in short, resident hacker

## Hello Jupyter

### pip install jupyter

 * Initially IPython Notebooks
   * interactive coding
   * instant feedback
 * Then rebranded to Jupyter
   * de'facto tool for a data scientist
 * Supports different *kernels*
   * R
   * nodejs
   * julia
   * Go
   * ...

#### Basic concepts

 * Organized into *cells*
 * *Cell* can be *code* or *markdown*
 * Cell is executed by *kernel*
 * JupyterLab is like IDE

#### Installing

```
pip install jupyter jupyterlab
```

#### Starting it up

```
(general) ➜  suricata-analytics-1 git:(next-suricon-2022-10-28) ✗ jupyter lab
[I 2022-10-30 06:10:48.141 ServerApp] jupyterlab | extension was successfully linked.
[I 2022-10-30 06:10:48.150 ServerApp] nbclassic | extension was successfully linked.
[I 2022-10-30 06:10:48.170 LabApp] JupyterLab extension loaded from /home/markus/venvs/general/lib/python3.10/site-packages/jupyterlab
[I 2022-10-30 06:10:48.170 LabApp] JupyterLab application directory is /home/markus/venvs/general/share/jupyter/lab
[I 2022-10-30 06:10:48.173 ServerApp] jupyterlab | extension was successfully loaded.
[I 2022-10-30 06:10:48.177 ServerApp] nbclassic | extension was successfully loaded.
[I 2022-10-30 06:10:48.177 ServerApp] The port 8888 is already in use, trying another port.
[I 2022-10-30 06:10:48.178 ServerApp] Serving notebooks from local directory: /home/markus/Projects/SN/suricata-analytics-1
[I 2022-10-30 06:10:48.178 ServerApp] Jupyter Server 1.21.0 is running at:
[I 2022-10-30 06:10:48.178 ServerApp] http://localhost:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
[I 2022-10-30 06:10:48.178 ServerApp]  or http://127.0.0.1:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
[I 2022-10-30 06:10:48.178 ServerApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 2022-10-30 06:10:48.216 ServerApp]

    To access the server, open this file in a browser:
        file:///home/markus/.local/share/jupyter/runtime/jpserver-395207-open.html
    Or copy and paste one of these URLs:
        http://localhost:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
     or http://127.0.0.1:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
Opening in existing browser session.
```

#### Code

It is suricon, so let's start the demo by downloading a PCAP file. With **pure python**. Purpose of this is to demo:

* Simple python code in notebook;
* To get initial input for next *slides*

Firstly, import supporting libraries.

In [15]:
import requests
from zipfile import ZipFile

Then define download link and output path as variables.

In [18]:
URL = "https://malware-traffic-analysis.net/2022/01/03/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap.zip"
OUTPUT = "/tmp/malware-pcap.zip"

Download and store the file. Notice the real-time output as code gets evalutated.

In [19]:
response = requests.get(URL, stream=True)
if response.status_code == 200:
    print("Download good, writing %d KBytes to %s" % 
          (int(response.headers.get("Content-length")) / 1024,
           OUTPUT))
    with open(OUTPUT, 'wb') as f:
        f.write(response.raw.read())
    print("Done")
else:
    print("Demo effect has kicked in")

Download good, writing 1254 KBytes to /tmp/malware-pcap.zip
Done


Then unzip the archive.

In [20]:
file_name = OUTPUT
with ZipFile(file_name, "r") as zip:
    zip.extractall(path="/tmp", pwd="infected".encode("utf-8"))

Find the PCAP and store for later use.

In [23]:
import glob
FILES = glob.glob("/tmp/*.pcap")
FILES

['/tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap']

In [40]:
print(FILES[0])

/tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap


#### Invoking a Shell command

* Writing code to do some simple things can be a hassle
* Jupyter provides some helpers
    * `%` calls builtin magic commands
    * `!` invokes any shell command

For example, we need a Suricata ruleset to proceed with presentation.

In [32]:
%pip install suricata-update

Note: you may need to restart the kernel to use updated packages.


In [33]:
!/home/jovyan/.local/bin/suricata-update

30/10/2022 -- 04:49:50 - <Info> -- Using data-directory /var/lib/suricata.
30/10/2022 -- 04:49:50 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
30/10/2022 -- 04:49:50 - <Info> -- Using /opt/suricata/share/suricata/rules for Suricata provided rules.
30/10/2022 -- 04:49:50 - <Info> -- Found Suricata version 7.0.0-beta1 at /opt/suricata/bin/suricata.
30/10/2022 -- 04:49:50 - <Info> -- Loading /etc/suricata/suricata.yaml
30/10/2022 -- 04:49:50 - <Info> -- Disabling rules for protocol pgsql
30/10/2022 -- 04:49:50 - <Info> -- Disabling rules for protocol modbus
30/10/2022 -- 04:49:50 - <Info> -- Disabling rules for protocol dnp3
30/10/2022 -- 04:49:50 - <Info> -- Disabling rules for protocol enip
30/10/2022 -- 04:49:50 - <Info> -- No sources configured, will use Emerging Threats Open
30/10/2022 -- 04:49:50 - <Info> -- Last download less than 15 minutes ago. Not downloading https://rules.emergingthreats.net/open/suricata-7.0.0/emerging.rules.tar.gz.
30/10/2022 -- 04:49:

In [38]:
!rm -rf /tmp/logs && mkdir /tmp/logs

In [46]:
!suricata -S /var/lib/suricata/rules/suricata.rules -l /tmp/logs -r /tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap -v

30/10/2022 -- 04:56:14 - <Notice> - This is Suricata version 7.0.0-beta1 RELEASE running in USER mode
30/10/2022 -- 04:56:14 - <Info> - CPUs/cores online: 12
30/10/2022 -- 04:56:14 - <Info> - fast output device (regular) initialized: fast.log
30/10/2022 -- 04:56:14 - <Info> - eve-log output device (regular) initialized: eve.json
30/10/2022 -- 04:56:14 - <Info> - stats output device (regular) initialized: stats.log
30/10/2022 -- 04:56:19 - <Info> - 1 rule files processed. 28761 rules successfully loaded, 0 rules failed
30/10/2022 -- 04:56:19 - <Info> - Threshold config parsed: 0 rule(s) found
30/10/2022 -- 04:56:19 - <Info> - 28764 signatures processed. 1183 are IP-only rules, 5166 are inspecting packet payload, 22211 inspect application layer, 108 are decoder event only
30/10/2022 -- 04:56:27 - <Info> - Starting file run for /tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap
30/10/2022 -- 04:56:27 - <Notice> - Threads created -> RX: 1 W: 12 FM: 1 FR: 1   Engine started.
3

## Meercat on Jupyter

### Import pandas as pd

* `pandas` is a python library that provides *dataframes*
* more than a library, it's actually a language by itself
* think R and Julia
* forget what you know about for loops
  * but it's totally worth it!

In [47]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
import numpy as np

In [52]:
import json

with open("/tmp/logs/eve.json", "r") as handle:
    DATA = [json.loads(line) for line in handle]
    DF = pd.json_normalize(DATA)

In [55]:
DF

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,2022-01-01T00:04:26.529009+0000,7.368444e+14,34.0,http,64.225.75.232,33556.0,198.71.247.91,80.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:05:53.595328+0000,5.277809e+14,57.0,http,219.84.187.214,56903.0,198.71.247.91,80.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:04:26.529078+0000,7.368444e+14,36.0,fileinfo,198.71.247.91,80.0,64.225.75.232,33556.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:25:53.420609+0000,5.096138e+14,202.0,http,162.158.88.141,47958.0,198.71.247.91,80.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:00:13.076985+0000,1.756928e+15,NaN,flow,18.212.79.210,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25896,2022-01-01T00:00:13.076985+0000,1.630767e+15,NaN,flow,218.208.118.2,52073.0,198.71.247.91,445.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25897,2022-01-01T00:00:13.076985+0000,8.976554e+13,NaN,flow,45.134.26.45,58164.0,198.71.247.91,65380.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25898,2022-01-01T00:00:13.076985+0000,1.179594e+15,NaN,flow,45.146.166.116,46439.0,198.71.247.91,19346.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25899,2022-01-01T00:00:13.076985+0000,1.292452e+15,NaN,flow,193.3.53.6,40232.0,198.71.247.91,1962.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
len(DF)

25901

In [59]:
len(DF.columns.values)

557

In [60]:
len([c for c in list(DF.columns.values) if not c.startswith("stats")])

120

In [56]:
DF.head(5)

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,2022-01-01T00:04:26.529009+0000,7.368444e+14,34.0,http,64.225.75.232,33556.0,198.71.247.91,80.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:05:53.595328+0000,5.277809e+14,57.0,http,219.84.187.214,56903.0,198.71.247.91,80.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:04:26.529078+0000,7.368444e+14,36.0,fileinfo,198.71.247.91,80.0,64.225.75.232,33556.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:25:53.420609+0000,5.096138e+14,202.0,http,162.158.88.141,47958.0,198.71.247.91,80.0,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:00:13.076985+0000,1.756928e+15,NaN,flow,18.212.79.210,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
DF.describe()

,flow_id,pcap_cnt,src_port,dest_port,tx_id,http.status,http.length,fileinfo.size,fileinfo.tx_id,icmp_type,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
count,2.590000e+04,2754.000000,23535.000000,23535.000000,1207.000000,2406.000000,2425.000000,1233.000000,1233.000000,2367.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.137316e+15,21675.688453,42136.566093,14515.903760,4.279205,352.704904,219.074639,272.029197,5.257908,7.993240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,6.469058e+14,12319.540304,17984.469425,17838.774877,7.498972,88.451780,99.841086,556.123714,8.729893,0.232495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2.342168e+09,18.000000,0.000000,0.000000,0.000000,200.000000,0.000000,18.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,5.888754e+14,10546.750000,36100.500000,1700.000000,0.000000,200.000000,51.000000,138.000000,0.000000,8.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.130943e+15,23269.500000,48856.000000,7000.000000,0.000000,404.000000,275.000000,275.000000,0.000000,8.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.696583e+15,33560.000000,54812.000000,22080.000000,7.000000,404.000000,279.000000,279.000000,9.000000,8.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2.251779e+15,39193.000000,65531.000000,65528.000000,41.000000,408.000000,316.000000,6138.000000,41.000000,8.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Threat hunting

### Advanced Analytics

### Ruleset analysis